In [2]:
import numpy as np
import pickle
import matplotlib.pyplot as plt
from plotly.offline import init_notebook_mode
init_notebook_mode(connected = True)

In [3]:
import os
experiment_result = {}
for file in os.listdir("experiments"):
    if file.endswith(".pickle"):
        with open(f"experiments/{file}","rb") as f:
            data = pickle.load(f)
        f.close()
        experiment_result[file.split(".")[0]] = data

In [10]:
experiment_result["binomial_ftm_rflvm"]["X"].shape

(1000, 2332, 2)

In [4]:
import numpy as np
from scipy.linalg import svd 
from typing import Tuple

def varimax(Phi, gamma = 1.0, q = 20, tol = 1e-6):

    p,k = Phi.shape
    R = np.eye(k)
    d=0
    for i in range(q):
        d_old = d
        Lambda = np.dot(Phi, R)
        u,s,vh = svd(np.dot(Phi.T,np.asarray(Lambda)**3 - (gamma/p) * np.dot(Lambda, np.diag(np.diag(np.dot(Lambda.T,Lambda))))))
        R = np.dot(u,vh)
        d = np.sum(s)
        if d_old!=0 and d/d_old < 1 + tol: break
    return np.dot(Phi, R)


def rotate_factors(player_factor_tensor:np.ndarray, use_varimax:bool = True)->Tuple[np.ndarray, np.ndarray]:
    """

    Args:
        player_factor_tensor (np.ndarray): sample x num factors x num players 
        varimax (bool): whether to apply varimax rotation or not

    Returns:
        np.ndarray: sample x num factors x num players  rotated tensor
    """

    n_samples, n_factors, _ = player_factor_tensor.shape
    output_tensor = np.zeros_like(player_factor_tensor)
    rotations = [np.eye(n_factors)]
    output_tensor[0,:,:] = player_factor_tensor[0,:,:] if not use_varimax else varimax(player_factor_tensor[0,:,:])
    for i in range(1,n_samples):
        U, _, V =  svd(output_tensor[0,:,:].dot(player_factor_tensor[i,:,:].T), full_matrices=False)
        rotation = U.dot(V)
        rotations.append(rotation)
        output_tensor[i,:,:] = rotation.dot(player_factor_tensor[i,:,:])
    print(len(rotations))
    return output_tensor, np.stack(rotations,axis = 0)




In [5]:
rotated_experiment_results = {}
# for model in experiment_result:
#     print(model, experiment_result[model]["X"])
#     rotated_experiment_results[model] = rotate_factors(np.swapaxes(experiment_result[model]["X"], axis1 = 1, axis2=2))

rotated_experiment_results["poisson_ast_rflvm"] = rotate_factors(np.swapaxes(experiment_result["poisson_ast_rflvm"]["X"],1,2))
rotated_experiment_results["poisson_stl_rflvm"] = rotate_factors(np.swapaxes(experiment_result["poisson_stl_rflvm"]["X"],1,2))
rotated_experiment_results["poisson_blk_rflvm"] = rotate_factors(np.swapaxes(experiment_result["poisson_blk_rflvm"]["X"],1,2))
rotated_experiment_results["poisson_dreb_rflvm"] = rotate_factors(np.swapaxes(experiment_result["poisson_dreb_rflvm"]["X"],1,2))
rotated_experiment_results["poisson_oreb_rflvm"] = rotate_factors(np.swapaxes(experiment_result["poisson_oreb_rflvm"]["X"],1,2))
rotated_experiment_results["gaussian_bpm_rflvm"] = rotate_factors(np.swapaxes(experiment_result["gaussian_bpm_rflvm"]["X"],1,2))
rotated_experiment_results["gaussian_dbpm_rflvm"] = rotate_factors(np.swapaxes(experiment_result["gaussian_dbpm_rflvm"]["X"],1,2))
rotated_experiment_results["gaussian_obpm_rflvm"] = rotate_factors(np.swapaxes(experiment_result["gaussian_obpm_rflvm"]["X"],1,2))
rotated_experiment_results["binomial_ftm_rflvm"] = rotate_factors(np.swapaxes(experiment_result["binomial_ftm_rflvm"]["X"],1,2))
rotated_experiment_results["binomial_fg2m_rflvm"] = rotate_factors(np.swapaxes(experiment_result["binomial_fg2m_rflvm"]["X"],1,2))
rotated_experiment_results["binomial_fg3m_rflvm"] = rotate_factors(np.swapaxes(experiment_result["binomial_fg3m_rflvm"]["X"],1,2))

1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000


In [6]:
rotated_means_results = {model : rotated_experiment_results[model][0].mean(axis=0).T for model in  rotated_experiment_results}

In [7]:
import pandas as pd
df = pd.read_csv("datasets/player_data.csv")

In [8]:
df = df.sort_values(by=["id","year"])


In [9]:
import plotly.express as px


def plot_scatter(dataframe, rotated_means_dict, metric, model, offset = "minutes"):
    key_name = f"{model}_{metric}_rflvm"
    df_size_vals = dataframe[[offset,"id"]].groupby("id").mean().reset_index()
    df_names = dataframe[["id","name"]].drop_duplicates()["name"].values
    data = pd.DataFrame(rotated_means_dict[key_name], columns=["x","y"])
    data["names"] = df_names
    data[offset] = df_size_vals[offset]
    if model == "binomial":
        df_color_vals = dataframe[[metric,offset, "id"]].groupby("id").sum().reset_index()
        data[f"{metric}_pct"] = df_color_vals[metric]/df_color_vals[offset]
        fig = px.scatter(data_frame= data,   
                   x = "x", y = "y", hover_data = [f"{metric}_pct",offset,"names"], title = metric, size = offset,
                         color = f"{metric}_pct", range_color = [.1,.5])
    elif model == "poisson":
        df_color_vals = dataframe[[metric,offset,"id"]].groupby("id").sum().reset_index()
        data[f"{metric}_rate"] = df_color_vals[metric]/df_color_vals[offset]
        fig = px.scatter(data_frame = data, x = "x", y = "y", hover_data = [f"{metric}_rate",offset,"names"], 
                         title = metric, size = offset,
                         color = f"{metric}_rate", range_color = [0,.2])
    elif model == "gaussian":
        df_color_vals = dataframe[[metric, "id"]].groupby("id").mean().reset_index()
        data["hover"] = df_color_vals[metric]
        fig = px.scatter(data_frame = data, x = "x", y = "y", color = "hover",
                    size = offset, hover_data = ["hover",offset,"names"], title = metric )
    
    fig.show()
    

    


In [10]:
plot_scatter(df, rotated_means_results, "fg2m", "binomial", "fg2a")

In [ ]:
plt.scatter(rotated_means[:,0], rotated_means[:,1], c= df_max_vals["fg3m"]/df_max_vals["fg3a"])
